# Is your house worth it's weight in gold?

Now we've calculated how much a given house rose in value over time - the question is, would you have been better off buying gold?

In the `data` folder, we have the daily price of one ounce of gold in USD, called `XAUUSD` in trading terms (e`X`change rate of `AU` aka Gold to `USD`)

We want to find out what would have happened if instead of buying a house, we would have bought an equivalent sum of gold. Finally we want to compare what the average profit would be if everyone had invested in gold instead of buying a house.

For simplicity, use the daily Closing price.

```{note}
Since gold is priced in USD, and our houseprices are in GBP, there is also a USD/GBP daily prices in the `data/fx` folder to convert between currencies.
```

Breaking it down:

1. Load daily gold prices into Iceberg - for bonus points, apply a yearly partitioning scheme
2. Load exchange rates into Iceberg - for bonus points, apply a yearly partitioning scheme
3. Convert daily gold prices from USD to GBP
4. For each row in our `profits` table, calculate the equivalent amount of gold they would have been able to purchase on that date
5. Calculate the total value of that amount of gold on the sell date
6. Calculate the profit of our gold trade
7. Compare to Gold profit with Housing profit

In [27]:
import polars as pl
from utils import catalog, engine
from pyiceberg.schema import Schema, NestedField
from pyiceberg.types import DecimalType, DateType, StringType
from pyiceberg.partitioning import PartitionSpec, YearTransform, PartitionField

## 1. Load daily gold prices into Iceberg
Start by creating the table in Iceberg. To organize things a bit better, I'm creating a new namespace `commodities` - we could imagine putting a table for oil prices or silver in here

In [2]:
catalog.create_namespace_if_not_exists("commodities")

gold_schema = Schema(
    NestedField(1, "date", DateType(), required=True, doc="Day of recorded price"),
    NestedField(
        2,
        "price",
        DecimalType(precision=38, scale=2),
        required=True,
        doc="Price in USD of one ounce of gold",
    ),
    identifier_field_ids=[1],
)

gold_prices_t = catalog.create_table_if_not_exists(
    "commodities.gold",
    schema=gold_schema,
    partition_spec=PartitionSpec(
        PartitionField(
            source_id=1, field_id=1, transform=YearTransform(), name="date_year"
        )
    ),
)

Next, read in the CSV, picking out the two columns we care about, remembering to convert to the correct schema, 

In [3]:
gold_prices = (
    pl.scan_csv("data/gold/daily_gold_prices.csv", separator=";", try_parse_dates=True)
    .select(pl.col("Date").alias("date"), pl.col("Close").alias("price"))
    .collect()
)

In [4]:
gold_prices

date,price
datetime[μs],f64
2004-06-11 00:00:00,384.1
2004-06-14 00:00:00,382.8
2004-06-15 00:00:00,388.6
2004-06-16 00:00:00,383.8
2004-06-17 00:00:00,387.6
…,…
2025-01-28 00:00:00,2763.17
2025-01-29 00:00:00,2759.68
2025-01-30 00:00:00,2794.06


In [5]:
gold_prices_t.append(gold_prices.to_arrow().cast(gold_schema.as_arrow()))

# 2. Load daily exchange rates into Iceberg

Similar process - create a new namespace and load the fx rates. Since this is technically a table of currency pairs, we can make the schema a bit more future-proof

In [8]:
catalog.create_namespace_if_not_exists("fx")

fx_schema = Schema(
    NestedField(1, "date", DateType(), required=True, doc="Day of recorded price"),
    NestedField(
        2,
        "from",
        StringType(),
        required=True,
        doc="From currency code of the currency pair",
    ),
    NestedField(
        3,
        "to",
        StringType(),
        required=True,
        doc="To currency code of the currency pair",
    ),
    NestedField(
        4,
        "exchange_rate",
        DecimalType(precision=38, scale=4),
        required=True,
        doc="Exchange rate of the currency pair",
    ),
    identifier_field_ids=[1, 2, 3],
)

Next, reading in the CSV file and transforming slightly to conform to our schema

In [9]:
fx_table = catalog.create_table_if_not_exists("fx.rates", schema=fx_schema)

In [11]:
fx_rates = (
    pl.scan_csv("data/fx/USD_GBP_clean.csv", try_parse_dates=True)
    .with_columns(pl.lit("USD").alias("from"), pl.lit("GBP").alias("to"))
    .select(
        pl.col("date"),
        pl.col("from"),
        pl.col("to"),
        pl.col("close").alias("exchange_rate"),
    )
    .collect()
)
fx_rates

date,from,to,exchange_rate
date,str,str,f64
2025-05-16,"""USD""","""GBP""",0.7529
2025-05-15,"""USD""","""GBP""",0.7516
2025-05-14,"""USD""","""GBP""",0.754
2025-05-13,"""USD""","""GBP""",0.7516
2025-05-12,"""USD""","""GBP""",0.759
…,…,…,…
2006-03-07,"""USD""","""GBP""",0.576
2006-03-06,"""USD""","""GBP""",0.5713
2006-03-03,"""USD""","""GBP""",0.5696


In [12]:
fx_table.append(fx_rates.to_arrow().cast(fx_schema.as_arrow()))

Next comes the business logic - the actual work of figuring out how much gold we could have bought and how much that would have sold for

In [19]:
sql = """
-- Convert USD gold prices into GBP denominated prices
with gold_prices as (
    select commodities.gold.date as gold_date, commodities.gold.price * fx.rates.exchange_rate as gold_price
    from commodities.gold
    join fx.rates on fx.rates.date = commodities.gold.date
), gold_purchase as (
-- Calculate how much gold (in ounces) the house owner could have bought at the time of purchase
    select address_id, 
    first_price / gold_price as purchased_gold
    from house_prices.profits
    join gold_prices on gold_date = house_prices.profits.first_day
), gold_sell as (
-- Calculate how much that amount of gold is worth at the time of sale, as well as the profit
    select house_prices.profits.address_id,
    (purchased_gold * gold_price) - first_price as gold_profit,
    profit as house_profit
    from house_prices.profits
    join gold_purchase on gold_purchase.address_id = house_prices.profits.address_id
    join gold_prices on gold_prices.gold_date = last_day
)
select * from gold_sell
"""

In [20]:
gold_vs_house_profits = pl.read_database(sql, engine)
gold_vs_house_profits

address_id,gold_profit,house_profit
str,"decimal[38,12]",i64
"""B1A3B4C54D4107EF29FBBD07BCBFAB…",29957.807806591744,5000
"""25455878C4728CC6284BD562A6181F…",110589.802268945110,46000
"""163D431B2AB454A98E15523629E1F2…",52955.744848463500,22000
"""949C688FC0E597FA9DFD678D2445F2…",70198.655947728248,115500
"""194BF7F0AEEA28D545C4A7534E7968…",167795.721945924350,75000
…,…,…
"""E10C90FCE21B3743A4F5077CE0C7CA…",260047.809414650717,85050
"""51F34D87CE575649A66069A2C0526A…",179841.650165830420,30005
"""A61CB99678109208C51EA48416DB6F…",542255.890232491500,575000


Now we have a per-address calculation, let's summarize the results

In [21]:
gold_vs_house_profits.select(pl.all().exclude("address_id")).describe()

statistic,gold_profit,house_profit
str,f64,f64
"""count""",942451.0,942451.0
"""null_count""",0.0,0.0
"""mean""",130688.660492,58374.959046
"""std""",174714.233629,255202.899074
"""min""",-917359.064101,-1.198e7
"""25%""",42116.252771,17500.0
"""50%""",87485.305817,40000.0
"""75%""",164702.335904,75000.0
"""max""",2.1051e7,2.23847e8


How many percent would have done better buying gold than a house?

In [33]:
with pl.Config(set_tbl_rows=100):
    summary_df = (
        gold_vs_house_profits.select(
            pl.col("house_profit")
            .sub(pl.col("gold_profit"))
            .qcut(100, labels=[f"Q{i + 1}" for i in range(100)], include_breaks=True)
        )
        .unnest("house_profit")
        .unique()
        .sort("breakpoint")
    )
summary_df

breakpoint,category
f64,cat
-604653.458806,"""Q1"""
-458657.896119,"""Q2"""
-385252.579736,"""Q3"""
-337413.912006,"""Q4"""
-303126.111512,"""Q5"""
-276526.137361,"""Q6"""
-255043.848988,"""Q7"""
-237040.490934,"""Q8"""
-221923.972903,"""Q9"""


# Exercise: What does it look like in your region?

This is the picture for all of the UK - what does it look like for your region?